In [ ]:
# pip install sql-metadata

In [2]:
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
from sql_metadata import Parser
import psycopg2
import psycopg2.extras

In [3]:
conn = psycopg2.connect("dbname=dvdrental host='192.168.16.16' user='ags14' port=5430 password='bitnine123!'")
conn.autocommit = True
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

In [40]:
# # extract raw sql-metadata tokens
# tables = Parser("""
# SELECT category.name, AVG(film.rental_rate) AS average_rental_rate
# FROM category
# JOIN film_category ON category.category_id = film_category.category_id
# JOIN film ON film_category.film_id = film.film_id
# GROUP BY category.name;
# """).tables
# (", ").join(tables)

'category, film_category, film'

In [4]:
# (", ").join(tables)

In [5]:
data = pd.read_csv('SQL_querylogs_dataset - dvdrental.csv')
data.head()

,query,action,detail,ref,쿼리\n작동 여부,쿼리 적합성,쿼리 처리 소요시간(ms)\n(최적화 전),쿼리 처리 소요시간(ms)\n(최적화 후),비고,Unnamed: 9,Unnamed: 10,최적화 전,최적화 후
0,"SELECT customer_id, COUNT(rental_id) AS total_...",고객별 총 대여 횟수 조회,rental 테이블에서 고객별로 rental_id를 카운트,rental,O,NaN,NaN,-,NaN,NaN,쿼리 처리 소요시간 평균(ms),NaN,NaN
1,"SELECT inventory.film_id, AVG(return_date - re...",각 영화의 평균 대여 기간 계산,rental 테이블과 inventory 테이블을 조인하여 영화별로 대여 기간의 평균...,"rental, inventory",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
2,"SELECT customer_id, genre, MAX(genre_count) AS...",각 고객별로 가장 많이 대여한 장르 조회,RANK() 함수로 각 고객별로 장르 대여 횟수에 따라 순위를 매김. PARTIT...,"rental, inventory, film_category, category",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
3,SELECT EXTRACT(YEAR FROM rental_date) AS renta...,모든 고객의 대여 기록이 있는 월별 대여 횟수,rental 테이블에서 연도와 월을 추출하여 그 기간 동안의 총 대여 횟수를 집계,rental,O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
4,"SELECT film.film_id, title, SUM(amount) AS tot...",각 영화에 대한 총 수익 조회\n\n,"payment, rental, inventory, film 테이블을 조인하여 각 영...","payment, rental, inventory, film",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN


In [6]:
sample_audit = pd.read_csv('sample_agaudit-2024-04-13-16-00-00.csv', header=None)
sample_audit.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,1,SELECT,2024-04-13 16:02:12 KST,3/330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
1,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,2,SELECT,2024-04-13 16:02:12 KST,3/331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
2,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,3,SELECT,2024-04-13 16:02:12 KST,3/332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
3,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,4,SELECT,2024-04-13 16:02:12 KST,3/333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
4,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,5,SELECT,2024-04-13 16:02:12 KST,3/334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement


In [ ]:
def trim(query):
    return query.replace('"'," ").replace("\n"," ").strip()

In [20]:
queries = [trim(i) for i in sample_audit[13]]
queries[:5]

['SELECT customer_id, COUNT(rental_id) AS total_rentals FROM rental GROUP BY customer_id;',
 'SELECT inventory.film_id, AVG(return_date - rental_date) AS average_rental_duration FROM rental JOIN inventory ON rental.inventory_id = inventory.inventory_id GROUP BY inventory.film_id;',
 'WITH GenreCounts AS (   SELECT      rental.customer_id,      category.name AS genre,      COUNT(*) AS genre_count,     MAX(COUNT(*)) OVER (PARTITION BY rental.customer_id) AS max_genre_count   FROM rental   JOIN inventory ON rental.inventory_id = inventory.inventory_id   JOIN film_category ON inventory.film_id = film_category.film_id   JOIN category ON film_category.category_id = category.category_id   GROUP BY rental.customer_id, category.name ) SELECT    customer_id,    genre,    genre_count AS max_genre_count FROM GenreCounts WHERE genre_count = max_genre_count;',
 'SELECT EXTRACT(YEAR FROM rental_date) AS rental_year, EXTRACT(MONTH FROM rental_date) AS rental_month, COUNT(rental_id) AS total_rentals FR

In [21]:
cooc_tables = []
for i in queries:
    tables = Parser(f"""{i}""").tables
    cooc_tables.append(tables)
cooc_tables[:5]

[['rental'],
 ['rental', 'inventory'],
 ['rental', 'inventory', 'film_category', 'category'],
 ['rental'],
 ['payment', 'rental', 'inventory', 'film']]

In [156]:
len(cooc_tables)

101

In [159]:
cursor.execute(f"""
                SET search_path TO ag_catalog;
                """)

In [162]:
def update_edge_count(table1, table2):
    cursor.execute(f"""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {{ table_name:'{table1}' }})-[r]-(v2:table {{ table_name:'{table2}' }})
                  SET r.count = r.count + 1
                  RETURN r.count 
                  $$) as (count agtype);
                  """)
    updated_count = int(cursor.fetchall()[0]['count'])
    print(f'edge count between {table1} and {table2} has been updated from {updated_count-1} to {updated_count}')

In [163]:
for n, cooc_table in enumerate(cooc_tables):
    print(f'\nquery {n}')
    cooc_table_len = len(cooc_table)
    try:
        if cooc_table_len <= 1:
            print('No relation are found')
            continue
        else:
            num_rel = cooc_table_len-1
            print(f'num_relation: {num_rel}')
            range_num_rel = range(num_rel)
            for idx in range_num_rel:
                table1, table2 = cooc_table[idx], cooc_table[idx+1]
                print(f'\njoined tables: {table1}, {table2}')
                update_edge_count(table1, table2)
    except IndexError:
        continue


query 0
No relation are found

query 1
num_relation: 1

joined tables: rental, inventory
edge count between rental and inventory has been updated from 2 to 3

query 2
num_relation: 3

joined tables: rental, inventory
edge count between rental and inventory has been updated from 3 to 4

joined tables: inventory, film_category

query 3
No relation are found

query 4
num_relation: 3

joined tables: payment, rental
edge count between payment and rental has been updated from 0 to 1

joined tables: rental, inventory
edge count between rental and inventory has been updated from 4 to 5

joined tables: inventory, film
edge count between inventory and film has been updated from 0 to 1

query 5
num_relation: 2

joined tables: rental, inventory
edge count between rental and inventory has been updated from 5 to 6

joined tables: inventory, film
edge count between inventory and film has been updated from 1 to 2

query 6
num_relation: 3

joined tables: film, inventory
edge count between film and inv

In [134]:
cursor.execute(f"""
                SET search_path TO ag_catalog;
                """)

In [93]:
cursor.execute("""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {table_name:'rental'})-[r]->(v2:table {table_name:'inventory'}) 
                  RETURN r.count 
                  $$) as (count agtype);
                """)
output = int(cursor.fetchall()[0]['count'])
output

3

In [91]:
cursor.execute("""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {table_name:'rental'})<-[r:ref]-(v2:table {table_name:'inventory'}) 
                  SET r.count = 13
                  RETURN r.count 
                  $$) as (count agtype);
                """)
output = int(cursor.fetchall()[0]['count'])
output

IndexError: list index out of range

In [114]:
def update_edge_count(table1, table2):
    cursor.execute(f"""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {{ table_name:'{table1}' }})-[r]-(v2:table {{ table_name:'{table2}' }}) 
                  RETURN r.count
                  $$) as (count agtype);
                  """)
    old_count = int(cursor.fetchall()[0]['count'])
    updated_count = old_count + 1
    cursor.execute(f"""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {{ table_name:'{table1}' }})-[r]-(v2:table {{ table_name:'{table2}' }})
                  SET r.count = {updated_count}
                  RETURN r.count 
                  $$) as (count agtype);
                  """)
    updated_check = int(cursor.fetchall()[0]['count'])
    print(f'  edge count between {table1} and {table2} has been updated from {old_count} to {updated_count}')

In [83]:
update_edge_count('city', 'address')

edge count between city and address has been updated from 15 to 16


In [ ]:
SELECT * FROM cypher('graph_v2',$$
MATCH (v1:table)<-[r:fkey]-(v2:table) 
RETURN r.count 
$$) as (count agtype);

In [ ]:
# def update_edge_count(table1, table2):
#     cursor.execute(f"""
#                   SELECT * FROM cypher('graph_v2',$$
#                   MATCH (v1:table {{ table_name:'{table1}' }})-[r]-(v2:table {{ table_name:'{table2}' }}) 
#                   RETURN r.count
#                   $$) as (count agtype);
#                   """)
#     old_count = int(cursor.fetchall()[0]['count'])
#     updated_count = old_count + 1
#     cursor.execute(f"""
#                   SELECT * FROM cypher('graph_v2',$$
#                   MATCH (v1:table {{ table_name:'{table1}' }})-[r]-(v2:table {{ table_name:'{table2}' }})
#                   SET r.count = {updated_count}
#                   RETURN r.count 
#                   $$) as (count agtype);
#                   """)
#     updated_check = int(cursor.fetchall()[0]['count'])
#     print(f'edge count between {table1} and {table2} has been updated from {old_count} to {updated_count}')